In [1]:
import numpy as np
import meshio
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import cg
from tqdm import tqdm


def create_mesh(x_points, y_points, length, height):
    nodes = np.zeros((x_points * y_points, 2))

    for j in range(y_points):
        for i in range(x_points):
            nodes[i + j * x_points] = [i * length / (x_points - 1), j * height / (y_points - 1)]

    elements = []
    for j in range(y_points - 1):
        for i in range(x_points - 1):
            elements.append([i + j * x_points, i + 1 + j * x_points, i + x_points + j * x_points])
            elements.append([i + 1 + j * x_points, i + x_points + 1 + j * x_points, i + x_points + j * x_points])

    return meshio.Mesh(nodes, [("triangle", np.array(elements))])

print("BUILDING STIFFNESS MATRIX")

# Calculate shape function derivatives
def shape_function_derivatives(coords):
    A_inv = np.linalg.inv(np.column_stack((coords, np.ones(3))))
    dN_dx = A_inv[0, :3]
    dN_dy = A_inv[1, :3]

    return dN_dx, dN_dy

def assemble_stiffness_matrix(mesh, young_modulus, poisson_ratio, thickness):
    nodes = np.array(mesh.points[:, :2])  # Convert nodes to CuPy array
    elements = np.array(mesh.cells_dict['triangle'])  # Convert elements to CuPy array
    num_nodes = len(nodes)
    
    # Plane stress constitutive matrix
    C = young_modulus / (1 - poisson_ratio**2) * np.array([
        [1, poisson_ratio, 0],
        [poisson_ratio, 1, 0],
        [0, 0, (1 - poisson_ratio) / 2]
    ])

    K = np.zeros((2 * num_nodes, 2 * num_nodes))
    
    for el_nodes in tqdm(elements, desc="Assembling stiffness matrix"):
        el = np.array(el_nodes, dtype=int)  # Ensure el is a CuPy array of integers
        # Get element nodal coordinates
        element_coords = nodes[el]
        
        # Calculate element area
        A = 0.5 * np.linalg.det(np.column_stack((element_coords, np.ones(3))))
        
        # Calculate the Jacobian matrix
        J = np.column_stack((element_coords[1] - element_coords[0], element_coords[2] - element_coords[0]))
        J_inv = np.linalg.inv(J)

        dN_dx, dN_dy = shape_function_derivatives(element_coords)
        
        # Calculate shape function derivatives in the reference element
        B_matrix = np.zeros((3, 6))
        B_matrix[0, 0::2] = dN_dx
        B_matrix[1, 1::2] = dN_dy
        B_matrix[2, 0::2] = dN_dy
        B_matrix[2, 1::2] = dN_dx
        
        # Calculate element stiffness matrix
        k_el = thickness * A * B_matrix.T @ C @ B_matrix
        
        # Assemble global stiffness matrix
        dof_map = np.repeat(2 * el, 2) + np.tile([0, 1], 3)
        K[np.ix_(dof_map, dof_map)] += k_el
    
    return K


def apply_boundary_conditions(K, nodes, tol=1e-6):
    fixed_nodes = np.where(np.abs(nodes[:, 0]) < tol)[0]
    fixed_dofs = np.hstack([2 * fixed_nodes, 2 * fixed_nodes + 1])
    free_dofs = np.setdiff1d(np.arange(2 * len(nodes)), fixed_dofs)
    
    return K[np.ix_(free_dofs, free_dofs)], fixed_dofs, free_dofs

    K_reduced, fixed_dofs, free_dofs = apply_boundary_conditions(K, mesh.points[:, :2])


def apply_load(nodes, load, free_dofs):
    length = nodes[-1, 0]
    free_end_nodes = np.where(np.abs(nodes[:, 0] - length) < 1e-6)[0]
    F = np.zeros(2 * len(nodes))
    F[2 * free_end_nodes + 1] = -load / len(free_end_nodes)
    return F[free_dofs]


print("INPUTTING PARAMETRIC DATA")
length = 24  # inches
width = 1  # inches
thickness = 1  # inches
young_modulus = 28e6  # psi
poisson_ratio = 0.3
load = 50   # pounds
x_points = 200
y_points = 20
mesh = create_mesh(x_points, y_points, length, width)

print("BUILDING K MATRIX")
K = assemble_stiffness_matrix(mesh, young_modulus, poisson_ratio, thickness)

print("APPLYING BOUNDARY CONDITIONS")
K_reduced, fixed_dofs, free_dofs = apply_boundary_conditions(K, mesh.points[:, :2])

print("K MATRIX SHAPE")
print(K.shape)

print("APPLYING LOADS")
F = apply_load(mesh.points[:, :2], load, free_dofs)

print("SOLVING FOR DISPLACEMENTS")
displacements_reduced, _ = cg(csr_matrix(K_reduced), F, tol=1e-9)
displacements = np.zeros(2 * len(mesh.points))
with tqdm(total=len(free_dofs)) as pbar:
    for i, dof in enumerate(free_dofs):
        displacements[dof] = displacements_reduced[i]
        pbar.update(1)


# Calculate von Mises stress at element centroids
def von_mises_stress(mesh, displacements, young_modulus, poisson_ratio):
    nodes = mesh.points[:, :2]
    elements = mesh.cells_dict['triangle']

    # Plane stress constitutive matrix
    C = young_modulus / (1 - poisson_ratio**2) * np.array([
        [1, poisson_ratio, 0],
        [poisson_ratio, 1, 0],
        [0, 0, (1 - poisson_ratio) / 2]
    ])

    stress = np.zeros(len(mesh.cells_dict["triangle"]))
    for idx, element in enumerate(mesh.cells_dict["triangle"]):
        element_coords = np.array([nodes[node_idx][:2] for node_idx in element])
        element_disp = np.array([displacements[2*node_idx:2*node_idx+2] for node_idx in element]).flatten()

        dN_dx, dN_dy = shape_function_derivatives(element_coords)

        B_matrix = np.zeros((3, 6))
        B_matrix[0, 0::2] = dN_dx
        B_matrix[1, 1::2] = dN_dy
        B_matrix[2, 0::2] = dN_dy
        B_matrix[2, 1::2] = dN_dx

        # Calculate element area
        A = 0.5 * np.abs(np.linalg.det(np.column_stack((element_coords, np.ones(3)))))

        A_inv = np.linalg.inv(np.column_stack((element_coords, np.ones(3))))
        B_matrix[0, 0::2] = A_inv[0, :3]
        B_matrix[1, 1::2] = A_inv[1, :3]
        B_matrix[2, 0::2] = A_inv[1, :3]
        B_matrix[2, 1::2] = A_inv[0, :3]

        # Calculate element strains
        global_displacements = displacements[np.repeat(2 * element, 2) + np.tile([0, 1], 3)]
        strain = B_matrix @ global_displacements

        # Calculate element stresses
        stress_el = C @ strain

        # Calculate von Mises stress
        stress[idx] = np.sqrt(stress_el[0]**2 - stress_el[0]*stress_el[1] + stress_el[1]**2 + 3*stress_el[2]**2)

    return stress


print("CALCULATING VON MISES STRESS")
stress = von_mises_stress(mesh, displacements, young_modulus, poisson_ratio)
print(f"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

#sanity check via classical analysis
# Input variables
F = load # Load applied at the free end (lb)
L = length # Length of the bar (in)
b = width # Width of the bar (in)
h = thickness # Height of the bar (in)
E = young_modulus # Modulus of elasticity of stainless steel (psi)

# Calculate the moment of inertia
I = (b * h**3) / 12
# Calculate the deflection using the formula
delta = -(F * L**3) / (3 * E * I)
print(f"The FEA max deflection of the flat bar is {np.min(displacements[1::2]):.4f} inches.")
print(f"The hand-calc max deflection of the flat bar is {delta:.4f} inches.")
errorDefl = (np.min(displacements[1::2]) - delta) / delta
print(f"{errorDefl*100:.1f}% error from handcalc deflection")
print(f"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

# Calculate the bending moment
M = F * L
# Calculate the distance from the neutral axis to the outermost fiber
c = h / 2
# Calculate the normal stress at the outermost fiber
sigma_x = (M * c) / I
# Calculate the shear stress at the outermost fiber
tau_xy = (F * h) / (2 * I)
# Calculate the von Mises stress
sigma_vm = ((sigma_x**2) + 3*(tau_xy**2))**0.5
print(f"The FEA max von Mises stress in the flat bar is {np.max(stress):.1f} psi.")
print(f"The hand-calc max von Mises stress in the flat bar is {sigma_vm:.1f} psi.")

errorVM = (np.max(stress) - sigma_vm) / sigma_vm
print(f"{errorVM*100:.1f}% error from handcalc von Mises stress")

BUILDING STIFFNESS MATRIX
INPUTTING PARAMETRIC DATA
BUILDING K MATRIX


Assembling stiffness matrix: 100%|██████████| 7562/7562 [00:01<00:00, 5241.58it/s]


APPLYING BOUNDARY CONDITIONS
K MATRIX SHAPE
(8000, 8000)
APPLYING LOADS
SOLVING FOR DISPLACEMENTS


100%|██████████| 7960/7960 [00:00<00:00, 1326077.76it/s]


CALCULATING VON MISES STRESS
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The FEA max deflection of the flat bar is -0.0965 inches.
The hand-calc max deflection of the flat bar is -0.0987 inches.
-2.2% error from handcalc deflection
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The FEA max von Mises stress in the flat bar is 7279.7 psi.
The hand-calc max von Mises stress in the flat bar is 7218.7 psi.
0.8% error from handcalc von Mises stress
